Business question: What are the most significant predictors of student achievement on a school level?

In [20]:
# Import all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import seaborn as sn
#import functions as fn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score, roc_auc_score, plot_confusion_matrix, precision_score, recall_score 
#from category_encoders import CountEncoder

In [21]:
#View options
#'display.float_format', lambda x: '%8.0f' % x
pd.set_option('display.max_columns', None, 'display.max_rows', None)

# Data Preparation and Cleaning

In [22]:
#Dealing with characteristic and location data first (this is merge 1)
df = pd.read_csv('data/us_doe_school_characteristics.csv',encoding='cp1252')

In [23]:
df.shape

(99763, 20)

In [24]:
df1 = df.drop(['FIPST','ST','STATE_AGENCY_NO','UNION','SHARED_TIME','TITLEI_STATUS_TEXT','MAGNET_TEXT','NSLP_STATUS','NSLP_STATUS_TEXT','VIRTUAL_TEXT'], axis=1).set_index(['ST_SCHID'])

In [25]:
df1.index.dtype

dtype('O')

In [26]:
df1.head()

,SCHOOL_YEAR,STATENAME,SCH_NAME,ST_LEAID,LEAID,NCESSCH,SCHID,TITLEI_STATUS,VIRTUAL
ST_SCHID,,,,,,,,,
AL-101-0010,2020-2021,ALABAMA,Albertville Middle School,AL-101,100005,1.000050e+10,100870,SWELIGTGPROG,NOTVIRTUAL
AL-101-0020,2020-2021,ALABAMA,Albertville High School,AL-101,100005,1.000050e+10,100871,SWELIGTGPROG,NOTVIRTUAL
AL-101-0110,2020-2021,ALABAMA,Albertville Intermediate School,AL-101,100005,1.000050e+10,100879,SWELIGTGPROG,NOTVIRTUAL
AL-101-0200,2020-2021,ALABAMA,Albertville Elementary School,AL-101,100005,1.000050e+10,100889,SWELIGTGPROG,NOTVIRTUAL
AL-101-0035,2020-2021,ALABAMA,Albertville Kindergarten and PreK,AL-101,100005,1.000050e+10,101616,SWELIGTGPROG,NOTVIRTUAL


In [27]:
locations_df = pd.read_csv('data/us_doe_locations.csv', encoding = 'cp1252')

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (14,15,21,22,39,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
locations_df.duplicated(['ST_SCHID']).sum()

0

In [29]:
df2 = locations_df[['ST_SCHID', 'LSTREET1','LCITY','LSTATE','LZIP','LEVEL']]

In [30]:
df2 = df2.set_index(['ST_SCHID'])

In [31]:
df1.shape

(99763, 9)

In [32]:
df2.shape

(101662, 5)

In [33]:
df2.head()

,LSTREET1,LCITY,LSTATE,LZIP,LEVEL
ST_SCHID,,,,,
AL-101-0010,600 E Alabama Ave,Albertville,AL,35950,Middle
AL-101-0020,402 E McCord Ave,Albertville,AL,35950,High
AL-101-0110,901 W McKinney Ave,Albertville,AL,35950,Middle
AL-101-0200,145 West End Drive,Albertville,AL,35950,Elementary
AL-101-0035,257 Country Club Rd,Albertville,AL,35951,Elementary


In [34]:
merged1 = df1.merge(df2, how='outer', left_index=True, right_index=True).reset_index()

In [35]:
merged1.shape

(101662, 15)

In [36]:
merged1.isna().sum()

ST_SCHID            0
SCHOOL_YEAR      1899
STATENAME        1899
SCH_NAME         1899
ST_LEAID         1899
LEAID            1899
NCESSCH          1899
SCHID            1899
TITLEI_STATUS    1899
VIRTUAL          1899
LSTREET1            3
LCITY               0
LSTATE              0
LZIP                0
LEVEL               0
dtype: int64

In [37]:
#About 2% of our data in this first merged dataframe contain nulls. Let's get rid of them
merged1 = merged1.dropna()

In [40]:
merged1['NCESSCH'] = merged1['NCESSCH'].astype(int)

In [44]:
enrolls = pd.read_csv('data/us_doe_enrollments.csv',encoding='cp1252')

In [45]:
enrolls.duplicated(['State School ID [Public School] 2020-21']).sum()

6

In [46]:
enrolls_dupes = enrolls.loc[enrolls.duplicated(['State School ID [Public School] 2020-21'])]

In [47]:
df4 = enrolls.drop_duplicates(subset = ['State School ID [Public School] 2020-21']).dropna()

In [48]:
df4.dtypes

ï»¿School Name                                                      object
State Name [Public School] Latest available year                    object
School ID - NCES Assigned [Public School] Latest available year    float64
State Name [Public School] 2020-21                                  object
School Name [Public School] 2020-21                                 object
Agency ID - NCES Assigned [Public School] Latest available year    float64
ANSI/FIPS State Code [Public School] Latest available year         float64
State Agency ID [Public School] 2020-21                             object
State School ID [Public School] 2020-21                             object
Total Students All Grades (Excludes AE) [Public School] 2020-21     object
Pupil/Teacher Ratio [Public School] 2020-21                         object
dtype: object

In [49]:
df4.shape

(99568, 11)

In [50]:
merged1.dtypes

ST_SCHID          object
SCHOOL_YEAR       object
STATENAME         object
SCH_NAME          object
ST_LEAID          object
LEAID            float64
NCESSCH            int64
SCHID            float64
TITLEI_STATUS     object
VIRTUAL           object
LSTREET1          object
LCITY             object
LSTATE            object
LZIP               int64
LEVEL             object
dtype: object

In [51]:
merged1.shape

(99760, 15)

In [52]:
enroll2 = df4[['State School ID [Public School] 2020-21','Total Students All Grades (Excludes AE) [Public School] 2020-21','Pupil/Teacher Ratio [Public School] 2020-21']]


In [54]:
enroll_final = enroll2.rename(columns={'State School ID [Public School] 2020-21': 'ST_SCHID', 'Total Students All Grades (Excludes AE) [Public School] 2020-21': 'Total Students', 'Pupil/Teacher Ratio [Public School] 2020-21':'Student:Teacher Ratio'})
                                       

In [55]:
enroll_final.head()

,ST_SCHID,Total Students,Student:Teacher Ratio
0,MA-0170-01700045,1139,11.89
1,NV-02-02093,352,35.2
2,NV-02-02364,163,16.3
3,WA-31025-1656,175,23.33
4,MN-010112-010112067,34,â€“


In [58]:
us_df = pd.merge(merged1, enroll_final, on='ST_SCHID',how='right')

In [61]:
maybe_final = us_df[['ST_SCHID','STATENAME','SCH_NAME','ST_LEAID','TITLEI_STATUS','VIRTUAL','LSTREET1','LCITY','LSTATE','LZIP','LEVEL','Total Students', 'Student:Teacher Ratio']]